In [ ]:
# from IPython.display import clear_output

# # install non-Python dependencies
# apt-get -y install graphviz graphviz-dev

# # Upgrade to TensorFlow 2.8
# pip install tensorflow==2.8 tensorflow-io==0.25.0 tfds-nightly pygraphviz

# # Install TensorFlow-GNN
# pip install tensorflow_gnn==0.2.0

# # Fix some dependencies
# pip install httplib2==0.20.4

# clear_output()

In [2]:
import pygraphviz as pgv
from tqdm import tqdm
from IPython.display import Image
import pandas as pd

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import tensorflow_gnn as tfgnn
import tensorflow_datasets as tfds

from tensorflow_gnn import runner
from tensorflow_gnn.models import gat_v2

print(f'Using TensorFlow v{tf.__version__} and TensorFlow-GNN v{tfgnn.__version__}')
print(f'GPUs available: {tf.config.list_physical_devices("GPU")}')

ImportError: DLL load failed while importing _graphviz: The specified module could not be found.

In [1]:
dat = pd.read_csv("C:\Users\Rajeshsingh\Desktop\folders\my\solar\solar_with_ai-main\solar_with_ai-main\out_clean.csv")
print("data shape:", dat.shape)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3308844045.py, line 1)

In [ ]:
dat.head()

,Date,Total Generation (KWH),Insolation,Grid Failure,Inverter,Module Cleaning,Rainy day,No Module Cleaning,Transformer replacement and maintenance,Cable and Fuse maintenance,Plant Shutdown,Internet,Battery,Cloudy,Module Cleaning by rains,NO-OFF-CLEAN MODULES
0,2012-07-06,3218.0,9.37536,1,0,0,0,0,0,0,0,0,0,0,0,0.0
1,2012-07-07,3633.0,9.37536,1,0,0,0,0,0,0,0,0,0,0,0,0.0
2,2012-07-08,4377.0,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
3,2012-07-09,4625.0,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
4,2012-07-10,648.0,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
# dat.d
x= dat.drop('Total Generation (KWH)', axis=1)
y=dat['Total Generation (KWH)']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
x_train

,Date,Insolation,Grid Failure,Inverter,Module Cleaning,Rainy day,No Module Cleaning,Transformer replacement and maintenance,Cable and Fuse maintenance,Plant Shutdown,Internet,Battery,Cloudy,Module Cleaning by rains,NO-OFF-CLEAN MODULES
585,2014-02-11,9.37536,1,0,0,0,0,0,0,0,0,0,0,0,0.0
2012,2018-01-08,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0
2268,2018-09-21,6.45700,0,0,1,0,0,0,0,0,0,0,0,0,774.0
1131,2015-08-11,9.37536,0,0,1,0,0,0,0,0,0,0,0,0,504.0
1591,2016-11-13,6.12000,1,1,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2264,2018-09-17,5.30800,0,0,0,0,0,0,0,0,0,0,1,0,0.0
2035,2018-01-31,9.37536,1,0,1,0,0,0,0,0,0,0,0,0,840.0
1697,2017-02-27,9.37536,1,0,1,0,0,0,0,0,0,0,0,0,702.0
3,2012-07-09,9.37536,0,1,0,0,0,0,0,0,0,0,0,0,0.0


In [ ]:
graph_schema_pbtxt = """
node_sets {
  key: "atom"
  value {
    description: "An atom in the molecule."

    features {
      key: "atom_features"
      value: {
        description: "[DATA] The features of the atom."
        dtype: DT_FLOAT
        shape { dim { size: 27 } }
      }
    }
  }
}

edge_sets {
  key: "bond"
  value {
    description: "A bond between two atoms in the molecule."
    source: "atom"
    target: "atom"

    features {
      key: "bond_features"
      value: {
        description: "[DATA] The features of the bond."
        dtype: DT_FLOAT
        shape { dim { size: 12 } }
      }
    }
  }
}

context {
  features {
    key: "toxicity"
    value: {
      description: "[LABEL] The toxicity class of the molecule (0 -> non-toxic; 1 -> toxic)."
      dtype: DT_INT64
    }
  }
  
  features {
    key: "molecule_id"
    value: {
      description: "[LABEL] The id of the molecule."
      dtype: DT_STRING
    }
  }
}
"""

In [ ]:
graph_schema = tfgnn.parse_schema(graph_schema_pbtxt)
graph_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)

In [ ]:
def make_graph_tensor(datapoint):
    """
    Convert a datapoint from the TF-DS CardioTox dataset into a `GraphTensor`.
    """
    # atom_mask is non-zero only for real atoms
    # [ V, ]
    atom_indices = tf.squeeze(tf.where(datapoint['atom_mask']), axis=1)
    
    # only keep features of real atoms
    # [ V, 27 ]
    atom_features = tf.gather(datapoint['atoms'], atom_indices)
    
    # restrict the bond mask to real atoms
    # [ V, V ]
    pair_mask = tf.gather(tf.gather(datapoint['pair_mask'], atom_indices, axis=0), atom_indices, axis=1)
    
    # restrict the bond features to real atoms
    # [ V, V, 12 ]
    pairs = tf.gather(tf.gather(datapoint['pairs'], atom_indices, axis=0), atom_indices, axis=1)
    
    # pair_mask is non-zero only for real bonds
    # [ E, 2 ]
    bond_indices = tf.where(pair_mask)
    
    # only keep features of real bonds
    # [ E, 12 ]
    bond_features = tf.gather_nd(pairs, bond_indices)
    
    # separate sources and targets for each bond
    # [ E, ]
    sources, targets = tf.unstack(tf.transpose(bond_indices))

    # active is [1, 0] for non-toxic molecules, [0, 1] for toxic molecules
    # [ ]
    toxicity = tf.argmax(datapoint['active'])
    
    # the molecule_id is included for reference
    # [ ]
    molecule_id = datapoint['molecule_id']

    # create a GraphTensor from all of the above
    atom = tfgnn.NodeSet.from_fields(features={'atom_features': atom_features},
                                     sizes=tf.shape(atom_indices))
    
    atom_adjacency = tfgnn.Adjacency.from_indices(source=('atom', tf.cast(sources, dtype=tf.int32)),
                                                  target=('atom', tf.cast(targets, dtype=tf.int32)))
    
    bond = tfgnn.EdgeSet.from_fields(features={'bond_features': bond_features},
                                     sizes=tf.shape(sources),
                                     adjacency=atom_adjacency)
    
    context = tfgnn.Context.from_fields(features={'toxicity': [toxicity], 'molecule_id': [molecule_id]})
    
    return tfgnn.GraphTensor.from_pieces(node_sets={'atom': atom}, edge_sets={'bond': bond}, context=context)


In [ ]:
train_dataset = dataset_splits['train'].map(make_graph_tensor)

NameError: ignored

In [ ]:
# train_data, test_data = [], []

# for _, group_data in dat.groupby("Total Generation (KWH)"):
#     random_selection = np.random.rand(len(group_data.index)) <= 0.5
#     train_data.append(group_data[random_selection])
#     test_data.append(group_data[~random_selection])

# train_data = pd.concat(train_data).sample(frac=1)
# test_data = pd.concat(test_data).sample(frac=1)

# print("Train data shape:", train_data.shape)
# print("Test data shape:", test_data.shape)

In [ ]:
# class_idx = {name: id for id, name in enumerate(dat.index)}
# feature_names = set(dat.columns) - {"Total Generation (KWH)"}
# num_features = len(feature_names)
# num_classes = len(class_idx)

In [ ]:
# hidden_units = [16, 16]
# learning_rate = 0.01
# dropout_rate = 0.5
# num_epochs = 300
# batch_size = 256

In [ ]:
# def run_experiment(model, x_train, y_train):
#     # Compile the model.
#     model.compile(
#         optimizer=keras.optimizers.Adam(learning_rate),
#         loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
#     )
#     # Create an early stopping callback.
#     early_stopping = keras.callbacks.EarlyStopping(
#         monitor="val_acc", patience=50, restore_best_weights=True
#     )
#     # Fit the model.
#     history = model.fit(
#         x=x_train,
#         y=y_train,
#         epochs=num_epochs,
#         batch_size=batch_size,
#         validation_split=0.15,
#         callbacks=[early_stopping],
#     )

#     return history

In [ ]:
# def display_learning_curves(history):
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

#     ax1.plot(history.history["loss"])
#     ax1.plot(history.history["val_loss"])
#     ax1.legend(["train", "test"], loc="upper right")
#     ax1.set_xlabel("Epochs")
#     ax1.set_ylabel("Loss")

#     ax2.plot(history.history["acc"])
#     ax2.plot(history.history["val_acc"])
#     ax2.legend(["train", "test"], loc="upper right")
#     ax2.set_xlabel("Epochs")
#     ax2.set_ylabel("Accuracy")
#     plt.show()

In [ ]:
# class_idx = {name: id for id, name in enumerate(dat['Total Generation (KWH)'])}

In [ ]:
# feature_names = set(dat.columns) - {"Date","Total Generation (KWH)"}
# num_features = len(feature_names)
# num_classes = len(class_idx)


# x_train = train_data[feature_names].to_numpy()
# x_test = test_data[feature_names].to_numpy()

# y_train = train_data["Total Generation (KWH)"]
# y_test = test_data["Total Generation (KWH)"]

# x_train = np.array([np.array(val) for val in x_train])
# x_test = np.array([np.array(val) for val in x_test])
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# time_stamp = np.array(list(dat.index)).astype('float64')

In [ ]:
# node_features = tf.cast(
#     dat.values('Total Generation (KWH)')[feature_names].to_numpy(), dtype=tf.dtypes.float32
# )

In [ ]:
# # Create an edges array (sparse adjacency matrix) of shape [2, num_edges].
# edges = dat[["Total Generation (KWH)"]].to_numpy().T
# # Create an edge weights array of ones.
# edge_weights = tf.ones(shape=edges.shape[1])

# # node_features = tf.cast(
# #     dat.sort_values('')[feature_names].to_numpy(), dtype=tf.dtypes.float32
# # )
# node_features = tf.cast(
#     dat.values('Total Generation (KWH)')[feature_names].to_numpy(), dtype=tf.dtypes.float32
# )
# # Create graph info tuple with node_features, edges, and edge_weights.
# graph_info = (node_features, edges, edge_weights)

# print("Edges shape:", edges.shape)
# print("Nodes shape:", node_features.shape)

In [ ]:
# class GraphConvLayer(layers.Layer):
#     def __init__(
#         self,
#         hidden_units,
#         dropout_rate=0.2,
#         aggregation_type="mean",
#         combination_type="concat",
#         normalize=False,
#         *args,
#         **kwargs,
#     ):
#         super().__init__(*args, **kwargs)

#         self.aggregation_type = aggregation_type
#         self.combination_type = combination_type
#         self.normalize = normalize

#         self.ffn_prepare = create_ffn(hidden_units, dropout_rate)
#         if self.combination_type == "gated":
#             self.update_fn = layers.GRU(
#                 units=hidden_units,
#                 activation="tanh",
#                 recurrent_activation="sigmoid",
#                 dropout=dropout_rate,
#                 return_state=True,
#                 recurrent_dropout=dropout_rate,
#             )
#         else:
#             self.update_fn = create_ffn(hidden_units, dropout_rate)

#     def prepare(self, node_repesentations, weights=None):
#         # node_repesentations shape is [num_edges, embedding_dim].
#         messages = self.ffn_prepare(node_repesentations)
#         if weights is not None:
#             messages = messages * tf.expand_dims(weights, -1)
#         return messages

#     def aggregate(self, node_indices, neighbour_messages, node_repesentations):
#         # node_indices shape is [num_edges].
#         # neighbour_messages shape: [num_edges, representation_dim].
#         # node_repesentations shape is [num_nodes, representation_dim]
#         num_nodes = node_repesentations.shape[0]
#         if self.aggregation_type == "sum":
#             aggregated_message = tf.math.unsorted_segment_sum(
#                 neighbour_messages, node_indices, num_segments=num_nodes
#             )
#         elif self.aggregation_type == "mean":
#             aggregated_message = tf.math.unsorted_segment_mean(
#                 neighbour_messages, node_indices, num_segments=num_nodes
#             )
#         elif self.aggregation_type == "max":
#             aggregated_message = tf.math.unsorted_segment_max(
#                 neighbour_messages, node_indices, num_segments=num_nodes
#             )
#         else:
#             raise ValueError(f"Invalid aggregation type: {self.aggregation_type}.")

#         return aggregated_message

#     def update(self, node_repesentations, aggregated_messages):
#         # node_repesentations shape is [num_nodes, representation_dim].
#         # aggregated_messages shape is [num_nodes, representation_dim].
#         if self.combination_type == "gru":
#             # Create a sequence of two elements for the GRU layer.
#             h = tf.stack([node_repesentations, aggregated_messages], axis=1)
#         elif self.combination_type == "concat":
#             # Concatenate the node_repesentations and aggregated_messages.
#             h = tf.concat([node_repesentations, aggregated_messages], axis=1)
#         elif self.combination_type == "add":
#             # Add node_repesentations and aggregated_messages.
#             h = node_repesentations + aggregated_messages
#         else:
#             raise ValueError(f"Invalid combination type: {self.combination_type}.")

#         # Apply the processing function.
#         node_embeddings = self.update_fn(h)
#         if self.combination_type == "gru":
#             node_embeddings = tf.unstack(node_embeddings, axis=1)[-1]

#         if self.normalize:
#             node_embeddings = tf.nn.l2_normalize(node_embeddings, axis=-1)
#         return node_embeddings

#     def call(self, inputs):
#         """Process the inputs to produce the node_embeddings.

#         inputs: a tuple of three elements: node_repesentations, edges, edge_weights.
#         Returns: node_embeddings of shape [num_nodes, representation_dim].
#         """

#         node_repesentations, edges, edge_weights = inputs
#         # Get node_indices (source) and neighbour_indices (target) from edges.
#         node_indices, neighbour_indices = edges[0], edges[1]
#         # neighbour_repesentations shape is [num_edges, representation_dim].
#         neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)

#         # Prepare the messages of the neighbours.
#         neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
#         # Aggregate the neighbour messages.
#         aggregated_messages = self.aggregate(
#             node_indices, neighbour_messages, node_repesentations
#         )
#         # Update the node embedding with the neighbour messages.
#         return self.update(node_repesentations, aggregated_messages)

In [ ]:
# class GNNNodeClassifier(tf.keras.Model):
#     def __init__(
#         self,
#         graph_info,
#         num_classes,
#         hidden_units,
#         aggregation_type="sum",
#         combination_type="concat",
#         dropout_rate=0.2,
#         normalize=True,
#         *args,
#         **kwargs,
#     ):
#         super().__init__(*args, **kwargs)

#         # Unpack graph_info to three elements: node_features, edges, and edge_weight.
#         node_features, edges, edge_weights = graph_info
#         self.node_features = node_features
#         self.edges = edges
#         self.edge_weights = edge_weights
#         # Set edge_weights to ones if not provided.
#         if self.edge_weights is None:
#             self.edge_weights = tf.ones(shape=edges.shape[1])
#         # Scale edge_weights to sum to 1.
#         self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

#         # Create a process layer.
#         self.preprocess = create_ffn(hidden_units, dropout_rate, name="preprocess")
#         # Create the first GraphConv layer.
#         self.conv1 = GraphConvLayer(
#             hidden_units,
#             dropout_rate,
#             aggregation_type,
#             combination_type,
#             normalize,
#             name="graph_conv1",
#         )
#         # Create the second GraphConv layer.
#         self.conv2 = GraphConvLayer(
#             hidden_units,
#             dropout_rate,
#             aggregation_type,
#             combination_type,
#             normalize,
#             name="graph_conv2",
#         )
#         # Create a postprocess layer.
#         self.postprocess = create_ffn(hidden_units, dropout_rate, name="postprocess")
#         # Create a compute logits layer.
#         self.compute_logits = layers.Dense(units=num_classes, name="logits")

#     def call(self, input_node_indices):
#         # Preprocess the node_features to produce node representations.
#         x = self.preprocess(self.node_features)
#         # Apply the first graph conv layer.
#         x1 = self.conv1((x, self.edges, self.edge_weights))
#         # Skip connection.
#         x = x1 + x
#         # Apply the second graph conv layer.
#         x2 = self.conv2((x, self.edges, self.edge_weights))
#         # Skip connection.
#         x = x2 + x
#         # Postprocess node embedding.
#         x = self.postprocess(x)
#         # Fetch node embeddings for the input node_indices.
#         node_embeddings = tf.gather(x, input_node_indices)
#         # Compute logits
#         return self.compute_logits(node_embeddings)

In [ ]:
# gnn_model = GNNNodeClassifier(
#     graph_info=graph_info,
#     num_classes=num_classes,
#     hidden_units=hidden_units,
#     dropout_rate=dropout_rate,
#     name="gnn_model",
# )

# # print("GNN output shape:", gnn_model([1, 10, 100]))

# # gnn_model.summary()

In [ ]:
# x_train = train_data.to_numpy()
# history = run_experiment(gnn_model, x_train, y_train)